In [30]:
import collections
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st
import scipy.special
import biocircuits
import tqdm
import utilities

import iqplot
import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

# <center> Homework 8.1: Cis-inhibition in the Delta-Notch System</center>
<hr>
<center><img src="__deltanotch.jpg" width="70%"></center>

\begin{align}
&\frac{\mathrm{d}N}{\mathrm{d}t} = \beta_N - \gamma_N N - k_t N D_\mathrm{trans} - k_c N D \\[1em]
&\frac{\mathrm{d}D}{\mathrm{d}t} = \beta_D - \gamma_D D - k_t N_\mathrm{trans} D - k_c N D \\[1em]
&\frac{\mathrm{d}S}{\mathrm{d}t} = k_t N D_\mathrm{trans} - \gamma_S S \\[1em]
&\frac{\mathrm{d}R}{\mathrm{d}t} = \beta_R\left(\frac{k_\mathrm{RS}\,k_t ND_\mathrm{trans}}{\gamma_S}\right)^p - \gamma_R R.
\end{align}

### <center> Parameters</center>
| Parameter     | Value     | Units                                    |
|:-------------:|:---------:|:----------------------------------------:|
| $\gamma_N$    | 0.08      | hours⁻¹                                  |
| $\gamma_D$    | 0.08      | hours⁻¹                                  |
| $\gamma_R$    | 0.01      | hours⁻¹                                  |
| $\gamma_S$    | 0.1       | hours⁻¹                                  |
| $k_t$         | 0.5       | (plate concentration units · hours)⁻¹    |
| $k_c$         | 5         | (relative fluorescent units · hours)⁻¹   |
| $k_{RS}$      | 6.67×10⁻⁴ | (relative fluorescent units · hours)⁻¹   |
| $\beta_N$     | 1         | relative fluorescent units/hour          |
| $\beta_R$     | 1.25×10⁸  | relative fluorescent units/hour          |
| $p$           | 2         | dimensionless                            |


## a) reporter concentration 🕵️‍♀️

In [31]:
def derivs_A(
    NDSR, t, 
    beta_N, beta_D, beta_R, 
    gamma_N, gamma_D, gamma_R, gamma_S, 
    k_c, k_t, k_RS,
    p,
    N_trans, D_trans
):
    N, D, S, R = NDSR
    dN_dt = beta_N - gamma_N*N - k_t*N*D_trans - k_c*N*D
    dD_dt = beta_D - gamma_D*D - k_t*N_trans*D - k_c*N*D
    dS_dt = k_t*N*D_trans - gamma_S*S
    dR_dt = beta_R*((k_RS*k_t*N*D_trans)/gamma_S)**p - gamma_R*R
    
    return np.array([dN_dt, dD_dt, dS_dt, dR_dt])

In [32]:
gamma_N = 0.08
gamma_D = 0.08
gamma_R = 0.01
gamma_S = 0.1
k_t = 0.5
k_c = 5
k_RS = 1 / 1500
beta_N = 1
beta_R = 1.25e8
p = 2

D_plate_array = np.array(
    [0.063, 0.084, 0.11, 0.15, 0.20, 0.26, 0.35, 0.46, 0.62, 0.82, 1.1, 1.4]
)

In [33]:
beta_D = 0
N_trans = 0

Ro = 0
Do = 200
So = 0      # ignoring this for now
t = np.linspace(0, 100, 1000)

In [34]:
t_sub = np.linspace(0, 30, 1000)
t_full = np.linspace(0, 100, 1000)
dict_R_traj_sub, dict_R_traj_full = {}, {}

for t, dict_R_traj in zip([t_full, t_sub], [dict_R_traj_full, dict_R_traj_sub]):
    for D_plate in D_plate_array: 
        D_trans = D_plate
        No = beta_N / (gamma_N + k_c * Do + k_t * D_plate)    
        NDSRo = np.array([No, Do, So, Ro])
        args = (beta_N, beta_D, beta_R, 
            gamma_N, gamma_D, gamma_R, gamma_S, 
            k_c, k_t, k_RS, p, 
            N_trans, D_trans)
        _NDSR = scipy.integrate.odeint(derivs_A, NDSRo, t, args)
        N_traj, D_traj, S_traj, R_traj = _NDSR.T    
        dict_R_traj[D_plate] = R_traj

In [35]:
palette_A = [
    '#13080a',
     '#2b090c',
     '#430b0e',
     '#5c0e14',
     '#7c1d2a',
     '#9b2b40',
     '#b73952',
     '#c74459',
     '#d64f61',
     '#e1636e',
     '#e68284',
     '#eaa299'
]

In [36]:
LOG_TOGGLE = pn.widgets.Toggle(name="LOG", width=350)
T_MAX_TOGGLE = pn.widgets.Toggle(name="t_max = 30", width=350, value=False)

@pn.depends(LOG_TOGGLE.param.value, T_MAX_TOGGLE.param.value)
def plotter_part_A(LOG, T_MAX):
    t = np.linspace(0, 30, 1000) if T_MAX else np.linspace(0, 100, 1000)
    dict_R_traj = dict_R_traj_sub if T_MAX else dict_R_traj_full
    
    y_axis_label = 'log(R)' if LOG else 'R'
    q = bokeh.plotting.figure(
        height=400, width=650, 
        title="[R] with varying Dplate",
        x_axis_label='time (hrs)', 
        y_axis_label=y_axis_label
    )

    _ = 0
    for D_plate, R_traj in dict_R_traj.items():
        _y = np.log(R_traj) if LOG else R_traj
        q.line(t, _y, line_width=3, line_color=palette_A[_])
        _ += 1

    items = [(f"D_plate: {D_plate}", [q.line(line_color=color, line_width=3)]) 
             for D_plate, color in zip(D_plate_array[::-1], palette_A[::-1])]
    legend = bokeh.models.Legend(items=items, location="center")
    q.add_layout(legend, 'right')

    return style(q)

lay_toggles = pn.Column(
    pn.Row(pn.Spacer(width=65), LOG_TOGGLE),
    pn.Row(pn.Spacer(width=65), T_MAX_TOGGLE),
)
pn.Column(plotter_part_A, lay_toggles)

Column
    [0] ParamFunction(function)
    [1] Column
        [0] Row
            [0] Spacer(width=65)
            [1] Toggle(name='LOG', width=350)
        [1] Row
            [0] Spacer(width=65)
            [1] Toggle(name='t_max = 30', width=350)

- From looking at $[R]$, we can really see how the cellular response to cis Delta is sharp, occuring at a fixed threshold independent of trans Delta (D_plate), whereas past the threshold, we see a graded response with respect to trans Delta (D_plate). Cool!  
- Mathematically, we can postulate that this arises from the absurdly ginormous $\beta_R$ term ($10^8$!) in dR/dt. This $\beta_R$ term scales $N D_{\mathrm{trans}}$. So as soon as the trans-activated complex begins building up in sufficiently noticeable concentrations, the reporter shoots off into unmediated bliss. 
- When we view the traces on a log-scale, we can visualize the threshold as the inflection point. Prior to the inflection point, we see that the growth rate dependence on D_plate looks very different because not enough cis complexes have been inactivated (this is mainly intuitive, but will be shown below.)

## complex analysis

Let's add two simple equations to our `derivs()` function to make it easier for us to plot the concentrations of inactive and active complexes together and see what kinds of behavior emerge at the threshold. (note that N and D are the *free* concentrations of unbound receptors and free ligands, and $R$ is scaled in an unfamiliar way.)

\begin{align}
\cfrac{\mathrm{d}\mathrm{CIS}}{\mathrm{d}t} &= k_c N D \\[0.5em]
\cfrac{\mathrm{d}\mathrm{TRANS}}{\mathrm{d}t} &= k_t N_{\mathrm{trans}} D + k_t N D_{\mathrm{trans}}
\end{align}

In [37]:
def derivs_A_supp(
    NDSRCT, t, 
    beta_N, beta_D, beta_R, 
    gamma_N, gamma_D, gamma_R, gamma_S, 
    k_c, k_t, k_RS,
    p,
    N_trans, D_trans
):
    N, D, S, R, C, T = NDSRCT
    dN_dt = beta_N - gamma_N*N - k_t*N*D_trans - k_c*N*D
    dD_dt = beta_D - gamma_D*D - k_t*N_trans*D - k_c*N*D
    dS_dt = k_t*N*D_trans - gamma_S*S
    dR_dt = beta_R*((k_RS*k_t*N*D_trans)/gamma_S)**p - gamma_R*R
    dC_dt = k_c*N*D
    dT_dt = k_c*N_trans*D + k_t*N*D_trans
    
    return np.array([dN_dt, dD_dt, dS_dt, dR_dt, dC_dt, dT_dt])

I have abstracted away redundant code to a module titled `utilities.py`; the code will also be provided below in the appendix. To interact with the code, you can add the `%load_ext autoreload`, `%autoreload 2` magic commands before the import line `import utilities`.

In [38]:
utilities.plotter_complexes()

Column
    [0] Row(align='center')
        [0] FloatSlider(end=1.4, name='D_plate', start=0.05, step=0.05, value=0.05, width=300)
    [1] ParamFunction(function)

- So we see that the trans activated complexes can only rise once the cis inactivated complexes stop growing, and that when playing with the trans Delta slider, it only changes the post-threshold slope, as we saw above...   
- *This shows that what I hypothesized above was not quite accurate. Even though $\beta_R$ is large, this threshold is inherently sharp purely by the structure of the circuit and interaction of N, D, and D_trans, not the largeness of one of the parameter values or the strange power function of R.*

Let's then think about what kinds of parameters can control the response time. 40 hours is quite long!  
Fixing D_plate = 0.5, we can observe the kinds of effects of scaling the Notch production rate and decay rate of both Notch and Delta. 

In [39]:
utilities.plotter_shift()

Column(align='center')
    [0] Row
        [0] Spacer(width=50)
        [1] FloatSlider(end=1.8, name='βN', start=0.5, value=1, width=150)
        [2] FloatSlider(end=0.3, name='γ', start=0.03, step=0.01, value=0.08, width=150)
    [1] ParamFunction(function)

- Increasing $\gamma$ means we reach the threshold faster; the cis inactivation levels off faster at a lower concentration.
- Increasing $\beta$ also shifts our curves slightly to the left; the cis inactivation levels off faster at a higher concentration. 
- But note that the production term remains a constant in the ODE's, whereas the decay term gets scaled by the instantaneous concentrations, so experimentally, for faster results, it might be worh trying some form of dilution or degradation to quicken the timescales of the reactions.

# b) two cells in contact ⛄️⛄️  🍻 

- Nondimensionalizing:

\begin{align}
\cfrac{\mathrm{d}N_1}{\mathrm{d}t} &= \beta_N - \gamma_N N_1 - k_c N_1 D_1 - k_t N_1 D_2 \\[0.5em]
\cfrac{\mathrm{d}N_2}{\mathrm{d}t} &= \beta_N - \gamma_N N_2 - k_c N_2 D_2 - k_t N_2 D_1 \\[0.5em]
\cfrac{\mathrm{d}D_1}{\mathrm{d}t} &= \beta_D^{(1)} - \gamma_D D_1 - k_c N_1 D_1 - k_t N_2 D_1\\[0.5em]
\cfrac{\mathrm{d}D_2}{\mathrm{d}t} &= \beta_D^{(2)} - \gamma_D D_2 - k_c N_2 D_2 - k_t N_1 D_2\\[0.5em]
\end{align}

$\hspace{15em}\mathrm{Nondimensionalizing... }\\[0.5em]$
\begin{align}
\cfrac{N_{1d}}{t_d}\cfrac{\mathrm{d}n_1}{\mathrm{d}t} &= \beta_N - \gamma_N N_{1d}n_1 - k_c N_{1d}D_{1d} n_1 d_1 - k_t N_{1d} D_{2d} n_1 d_2 \\[0.5em]
\cfrac{N_{2d}}{t_d}\cfrac{\mathrm{d}n_2}{\mathrm{d}t} &= \beta_N - \gamma_N N_{2d}n_2 - k_c N_{2d}D_{2d} n_2 d_2 - k_t N_{2d} D_{1d} n_2 d_1 \\[0.5em]
\cfrac{D_{1d}}{t_d}\cfrac{\mathrm{d}d_1}{\mathrm{d}t} &= \beta_D^{(1)} - \gamma_D D_{1d} d_1 - k_c N_{1d} D_{1d} n_1 d_1 - k_t N_{2d} D_{1d} n_2 d_1\\[0.5em]
\cfrac{D_{2d}}{t_d}\cfrac{\mathrm{d}d_2}{\mathrm{d}t} &= \beta_D^{(2)} - \gamma_D D_{2d} d_2 - k_c N_{2d} D_{2d} n_2 d_2 - k_t N_{1d} D_{2d} n_1 d_2\\[0.5em]
\end{align}

$\hspace{15em}\mathrm{Setting... }\\[0.5em]$
\begin{align}
\boxed{
\hspace{1.2em}t_d = \cfrac{1}{\gamma_N} = \cfrac{1}{\gamma_D} = \cfrac{1}{\gamma}  \\[0.5em]
N_{1d} = N_{2d} = D_{1d} = D_{2d} = \cfrac{\gamma}{k_t}  \\[0.5em]
}
\end{align}

\begin{align}
\\[0.5em]
\cfrac{\mathrm{d}n_1}{\mathrm{d}t} &= \beta_N \cfrac{k_t}{\gamma^2} - n_1 - k_c \cfrac{k_t}{\gamma^2} n_1 d_1 - n_1 d_2\\[0.5em]
\cfrac{\mathrm{d}n_2}{\mathrm{d}t} &= \beta_N \cfrac{k_t}{\gamma^2} - n_2 - k_c \cfrac{k_t}{\gamma^2} n_2 d_2 - n_2 d_1\\[0.5em]
\cfrac{\mathrm{d}d_1}{\mathrm{d}t} &= \beta_D^{(1)} \cfrac{k_t}{\gamma^2} - d_1 - k_c \cfrac{k_t}{\gamma^2} n_1 d_1 - n_2 d_1\\[0.5em]
\cfrac{\mathrm{d}d_2}{\mathrm{d}t} &= \beta_D^{(2)} \cfrac{k_t}{\gamma^2} - d_2 - k_c \cfrac{k_t}{\gamma^2} n_2 d_2 - n_1 d_2\\[0.5em]
\end{align}

$\hspace{15em}\mathrm{Setting... }\\[0.5em]$
\begin{align}
\boxed{
\hspace{0.6em} \beta_n = \beta_N \cfrac{k_t}{\gamma^2} \hspace{3em} \kappa = \cfrac{k_c k_t}{\gamma^2} \\[0.5em]
\beta_d^{(1)} = \beta_D^{(1)} \cfrac{k_t}{\gamma^2} \hspace{2em} \beta_d^{(2)} = \beta_D^{(2)} \cfrac{k_t}{\gamma^2}
}
\end{align}

\begin{align}
\boxed{
\cfrac{\mathrm{d}n_1}{\mathrm{d}t} = \beta_n - n_1 - \kappa n_1 d_1 - n_1 d_2\\[0.5em]
\cfrac{\mathrm{d}n_2}{\mathrm{d}t} = \beta_n - n_2 - \kappa n_2 d_2 - n_2 d_1\\[0.5em]
\cfrac{\mathrm{d}d_1}{\mathrm{d}t} = \beta_d^{(1)} - d_1 - \kappa n_1 d_1 - n_2 d_1\\[0.5em]
\cfrac{\mathrm{d}d_2}{\mathrm{d}t} = \beta_d^{(2)} - d_2 - \kappa n_2 d_2 - n_1 d_2\\[0.5em]
}
\end{align}

- Homogenous steady state:  
    Subtracting $\mathrm{d}d_2/\mathrm{d}t$ from $\mathrm{d}d_1/\mathrm{d}t$, and setting $n_1 = n_2 \equiv n_0, d_1 = d_2 \equiv d_0 $ we arrive at the following: 
\begin{align}
0 &= \beta_d^{(1)} - \beta_d^{(2)} - d_1 + d_2 - \kappa n_1 d_1 + \kappa n_2 d_2 - n_2 d_1 + n_1 d_2 \\[0.5em]
0 &= \beta_d^{(1)} - \beta_d^{(2)} - d_0 + d_0 - \kappa n_0 d_0 + \kappa n_0 d_0 - n_0 d_0 + n_0 d_0 \\[0.5em]
0 &= \beta_d^{(1)} - \beta_d^{(2)}
\end{align}
\begin{align}
\boxed{
\beta_d^{(1)} = \beta_d^{(2)}
}
\end{align}
    The steady states for the two cells can only be equivalent if Delta is produced at the same rate for both cells. At this homoegenous steady state, either both cells are receiving or both are sending. In order for one cell to signal to another, a difference in production rates $\beta_d^{(i)}$ of the delta ligand will suffice. The final concentrations must also satisfy the relation $\beta_n = n_0 (1+d_0(\kappa + 1))$. 

- Deriving amplification:

\begin{align}
\text{Amplification} &= \cfrac{1-S1/S2}{1-\beta_D^{(2)}/\beta_D^{(1)}} \\[1em]
\cfrac{\mathrm{d}S_1}{\mathrm{d}t} &= k_t N_1 D_2 - \gamma_S S_1 = 0\\[0.5em]
\cfrac{\mathrm{d}S_2}{\mathrm{d}t} &= k_t N_2 D_1 - \gamma_S S_2 = 0\\[0.5em]
S_{1, \mathrm{st}} &= \cfrac{k_t}{\gamma_S} N_1 D_2 \\[0.5em]
S_{2, \mathrm{st}} &= \cfrac{k_t}{\gamma_S} N_2 D_1 \\[1em]
\cfrac{S_1, \mathrm{st}}{S_2, \mathrm{st}} &= \cfrac{N_1 D_2}{N_2 D_1} \\[0.5em]
&\text{ since }N_{1d} = N_{2d} , D_{1d} = D_{2d} \\[0.5em]
&= \cfrac{n_1 d_2}{n_2 d_1} \hspace{2em} \\[0.5em]
\cfrac{\beta_D^{(2)}}{\beta_D^{(1)}} &= \cfrac{ (\gamma^2 / k_t) \beta_d^{(1)}}{ (\gamma^2 / k_t)  \beta_d^{(2)}} \\[0.5em]
&= \beta_d^{(2)} / \beta_d^{(2)} \\[1em]
\text{Amplification} &= \cfrac{1- n_1 d_2 / n_2 d_1}{1-\beta_d^{(2)}/\beta_d^{(1)}} \\[1em]
\end{align}

- Unique steady state:   
    It has driven me a little crazy to not know where the uniqueness comes from, but I have seen 1D systems proven using the Picard-Lindelöf theorem / Cauchy-Lipschitz theorem using successive Picard's iteration, and showing that the integrated terms form a convergent series using Grönwalls lemma. Due to time limitations, I did not have a chance to explore this further and convinced myself it was a dead end.
    I decided instead to plot the 2-dimensional set of equations we need to solve on desmos https://www.desmos.com/calculator/taynhlgtee:
\begin{align}
d_1 &= \cfrac {\beta_d^{(1)}} {1+\kappa\beta_n / (1+\kappa d_1 + d_2) + \beta_n / (1+\kappa d_2 + d_1)} \\[0.5em]
d_2 &= \cfrac {\beta_d^{(2)}} {1+\kappa\beta_n / (1+\kappa d_2 + d_1) + \beta_n / (1+\kappa d_1 + d_2)} \\[0.5em]
\end{align}
    We can see in the first quadrant, there can only ever be one crossing in the positive parameter space. I tried making a geometric argument, that $\mathrm{d}d_1/\mathrm{d}d_2$ and $\mathrm{d}d_2/\mathrm{d}d_1$ are both positive, are concave down functions of each other, and have positive intercepts. I believe this is a sufficient argument. As a proof of concept, I've graphed another example that satisfies these conditions with a simple square root function: https://www.desmos.com/calculator/rz7nbsvylq    
    I am 65% confident that this makes sense, but 100% sure that this is not at all how we were supposed to go about it.   
    To answer the actual question that is posed, knowing the uniqueness of the steady state means we only have to evaluate the nature of the stability at one point: it is either stable, unstable, or has limit cycles. There is no possibility of a toggle, or a separatrix/basins, etc. This also implies that the system is robust to initial conditions, that the stability of the system is determined by the parameters (production rates, decay rates, rate constants, etc. ) alone. 

- Linear stability:  
\begin{align}
A &= \begin{pmatrix}
-(1+\kappa d_1 + d_2) & 0 & -\kappa n_1 & - n_1 \\
0 & -(1+\kappa d_2 + d_1) & -n_2 & - \kappa n_2 \\
-\kappa d_1 & - d_1 & -(1+\kappa n_1+n_2) & 0 \\
-d_2 & -\kappa d_2 & 0 & -(1+\kappa n_2 + n_1)
\end{pmatrix} \\[0.5em]
A &= - \begin{pmatrix}
1+\kappa d_1 + d_2 & 0 & \kappa n_1 & n_1 \\
0 & 1+\kappa d_2 + d_1 & n_2 & \kappa n_2 \\
\kappa d_1 & d_1 & 1+\kappa n_1+n_2 & 0 \\
d_2 & \kappa d_2 & 0 & 1+\kappa n_2 + n_1
\end{pmatrix} \\[0.5em]
A^T &= - \begin{pmatrix}
1+\kappa d_1 + d_2 & 0 & \kappa d_1 & d_2 \\
0 & 1+\kappa d_2 + d_1 & d_1 & \kappa d_2 \\
\kappa n_1 & n_2 & 1+\kappa n_1+n_2 & 0 \\
n_1 & \kappa n_2 & 0 & 1+\kappa n_2 + n_1
\end{pmatrix} \\[0.5em]
&\hspace{5em} 1 + \kappa d_1 + d_2 > \kappa d_1 + d_2 \\[0.5em]
&\hspace{5em} 1 + \kappa d_2 + d_1 > \kappa d_2 + d_1 \\[0.5em]
&\hspace{5em} 1 + \kappa n_1 + n_2 > \kappa n_1 + n_2 \\[0.5em]
&\hspace{5em} 1 + \kappa n_2 + n_1 > \kappa n_2 + n_1 \\[0.5em]
\end{align}
    Looking at the transpose of the Jacobian, the diagonal elements are all 1 greater than the summation of the off-diagonals. By the Gerschgorin Circle Theorem, this means that none of the eigenvalues will be complex, and all the eigenvalues of the negative of $A^T$ will have positive real parts, which means hat $A^T$, and thus $A$ will have eigenvalues with negative real parts. Although we have not solved for the steady state directly, we can already tell that *at* steady state, the inequalities above give us local stability.

- Amplification  
    Note that we are using inverse rates relative to the paper, so our $k$ rate constants will be greater than unity.

In [40]:
def derivs_B(
    n1n2d1d2, t, 
    beta_n, beta_d1, beta_d2,
    kappa
):
    n1, n2, d1, d2 = n1n2d1d2
    dn1_dt = beta_n - n1 - kappa*d1*n1 - d2*n1
    dn2_dt = beta_n - n2 - kappa*d2*n2 - d1*n2
    dd1_dt = beta_d1 - d1 - kappa*d1*n1 - d1*n2
    dd2_dt = beta_d2 - d2 - kappa*d2*n2 - d2*n1
    
    return np.array([dn1_dt, dn2_dt, dd1_dt, dd2_dt])


def find_amplification(
    n1, n2, d1, d2, 
    beta_d1, beta_d2
):
    amplification = (1 - (d2*n1)/(d1*n2)) / (1 - beta_d2/beta_d1)
    return amplification

In [41]:
beta_n = 200

kappa_space = np.logspace(0, 3, 10)             # **************
beta_d1_space = np.linspace(0.01, 500, 150)          # **************

n1o, n2o, d1o, d2o = 3, 5, 100, 40             # **************
n1n2d1d2_o = np.array([n1o, n2o, d1o, d2o]) 

t = np.linspace(0, 10, 500)


dict_kappa_ampbeta = {}
for kappa in tqdm.tqdm(kappa_space[::-1]):
    amps = []
    for beta_d1 in beta_d1_space: 
        beta_d2 = beta_d1*1.35
        args = (beta_n, beta_d1, beta_d2, kappa)    
        _n1n2d1d2 = scipy.integrate.odeint(derivs_B, n1n2d1d2_o, t, args)
        n1_traj, n2_traj, d1_traj, d2_traj = _n1n2d1d2.T    

        n1_st, n2_st, d1_st, d2_st = n1_traj[-1], n2_traj[-1], d1_traj[-1], d2_traj[-1]
        amplification = find_amplification(n1_st, n2_st, d1_st, d2_st, beta_d1, beta_d2)
        amps.append(amplification)
    dict_kappa_ampbeta[kappa] = amps
    
beta_bar_space = beta_d1_space * 2.35 / 2

100%|██████████| 10/10 [00:08<00:00,  1.11it/s]


In [42]:
palette_B = [
     '#090e1c',
     '#19224f',
     '#294187',
     '#385ab1',
     '#475cb1',
     '#4a6dbb',
     '#4e7cc3',
     '#568acb',
     '#6999d7',
     '#7da9db'
]

In [43]:
LOG_TOGGLE_AMP = pn.widgets.Toggle(name="LOG", width=350, value=False)
@pn.depends(LOG_TOGGLE_AMP.param.value)
def plotter_amplification(LOG_TOGGLE_AMP):
    y_axis_type = "log" if LOG_TOGGLE_AMP else "linear"
    q = bokeh.plotting.figure(
        height=375, width=625, 
        title="Amplification vs. β1 varying κ",
        x_axis_label='β_bar', 
        y_axis_label='amplification',
        y_axis_type=y_axis_type
    )
    _ = 0
    for kappa, amps in dict_kappa_ampbeta.items():
        q.line(beta_bar_space, amps, line_width=3, color=palette_B[::-1][_])
        _ += 1
    items = [("κ: {:.1e}".format(kappa), [q.line(line_color=color, line_width=3)]) 
             for kappa, color in zip(kappa_space[::-1], palette_B[::-1])]
    legend = bokeh.models.Legend(items=items, location="center")
    q.add_layout(legend, 'right')
    q = style(q)

    return q
pn.Column(plotter_amplification, pn.Row(pn.Spacer(width=80), LOG_TOGGLE_AMP))

Column
    [0] ParamFunction(function)
    [1] Row
        [0] Spacer(width=80)
        [1] Toggle(name='LOG', width=350)

- Let's think about why the amplification sharply peaks. When there is slightly more $n$ in one cell and slightly more $d$ in its neighbor, the signalling begins to favor a particular direction. Thus, varying the production rates in the two cells can amplify signalling activity and formation of activted complexes in one of the cells relative to the other. 
- This effect peaks when $\bar{\beta}~= 200,$ or when $\beta_d^{(1)}+\beta_d^{(2)} = 400$. Thus when the ratio between the two are fixed at 1.35, we are only looking at how the numerator $d_2 n_1/d_1 n_2$ steady state metric is changing. 
- Increasing $\kappa$ boosts this effect; the survival times as well as the energy associated with the bound complexes are favored. When thinking about the energetics in this sense, we suspect that $\kappa$ less than unity will have the opposite effect because the degradation factors are faster than the survival times of the complexes. 

## c) NICD repression: homogeneous steady state bifurcation
\begin{align}
&\frac{\mathrm{d}n_1}{\mathrm{d}t} = \beta_n - n_1 - \kappa d_1 n_1 - d_2 n_1 \\[1em]
&\frac{\mathrm{d}n_2}{\mathrm{d}t} = \beta_n - n_2 - \kappa d_2 n_2 - d_1 n_2 \\[1em]
&\frac{\mathrm{d}d_1}{\mathrm{d}t} = \frac{\beta_d}{1 + s_1^\alpha} - d_1 - \kappa d_1 n_1 - d_1 n_2 \\[1em]
&\frac{\mathrm{d}d_2}{\mathrm{d}t} = \frac{\beta_d}{1 + s_2^\alpha} - d_2 - \kappa d_2 n_2 - d_2 n_1 \\[1em]
&\frac{\mathrm{d}s_1}{\mathrm{d}t} = \kappa_s d_2 n_1 - \gamma s_1\\[1em]
&\frac{\mathrm{d}s_2}{\mathrm{d}t} = \kappa_s d_1 n_2 - \gamma s_2.
\end{align}

So with our added hill functions, we can first think about looking for the fixed point by writing out the nullclines. For a homogeneous steady state, our system of equations reduce down to 3-dimensions. We can define the following: 
\begin{align}
n_0 \equiv n_{1, st} = n_{2, st} \\[0.5em]
d_0 \equiv d_{1, st} = d_{2, st} \\[0.5em]
s_0 \equiv s_{1, st} = s_{2, st} \\[0.5em]
\end{align}
Note that when solving for $s_0$, it is merely proportional to the product of $\kappa_s / \gamma d_0 n_0$, so when plotting nullclines we can limit our focus on $n_0$ and $d_0$ and rewrite them in terms of each other via substitution. 

\begin{align}
\hspace{0em} \text{Nullclines} :& \\[0.5em]
n_0 &= \cfrac{\beta_n}{1+d_0 (1+\kappa)} \\[0.5em]
d_0 &= \cfrac{\beta_d}{
(1+n_0 (1+\kappa))\left(1+\left[\cfrac{\kappa_s}{\gamma (1+\kappa)} (\beta_n-n_0)\right]^\alpha \right)
} 
\end{align}

Let's code this up, and then look at the trajectories. I have lifted the intersection-finding code from the pip package `intersection`. This, and the code for the plotter, are abstracted away into `utilities`.

In [44]:
def derivs_C(n1n2d1d2s1s2, t, alpha, beta_n, beta_d, gamma, kappa, kappa_s):
    n1, n2, d1, d2, s1, s2 = n1n2d1d2s1s2
    dn1_dt = beta_n - n1 - kappa*d1*n1 - d2*n1
    dn2_dt = beta_n - n2 - kappa*d2*n2 - d1*n2

    dd1_dt = beta_d/(1+s1**alpha) - d1 - kappa*d1*n1 - d1*n2
    dd2_dt = beta_d/(1+s2**alpha) - d2 - kappa*d2*n2 - d2*n1

    ds1_dt = kappa_s*d2*n1 - gamma*s1
    ds2_dt = kappa_s*d1*n2 - gamma*s2

    return np.array([dn1_dt, dn2_dt, dd1_dt, dd2_dt, ds1_dt, ds2_dt])

In [45]:
no, do = 5, 100
n1o, n2o = no, no
d1o, d2o = do, do
s1o, s2o = 0, 0


alpha = 2
beta_n = 8.6
beta_d = 18
gamma = 5
kappa = 10
kappa_s = 20

args = (alpha, beta_n, beta_d, gamma, kappa, kappa_s)

In [46]:
color_n1 = '#5779a3'
color_n2 = '#84b5b2'
color_d1 = '#d1605e'
color_d2 = '#e59244'
color_s1 = "#808BBE"
color_s2 = "#b09dc9"

In [47]:
utilities.plotter_homogeneous(
    color_n1=color_n1, color_n2=color_n2, 
    color_d1=color_d1, color_d2=color_d2,
    color_s1=color_s1, color_s2=color_s2
)

Row
    [0] Column
        [0] Spacer(height=10)
        [1] FloatSlider(end=10, name='βn', start=0.1, value=8.6, width=95)
        [2] IntSlider(end=30, name='βd', start=1, value=18, width=95)
        [3] FloatSlider(end=20, name='γ', start=1, step=1, value=5, width=95)
        [4] FloatSlider(end=10, name='κ', start=1, step=0.5, value=10, width=95)
        [5] FloatSlider(end=100, name='κS', start=1, step=2, value=20, width=95)
    [1] Column
        [0] ParamFunction(function)
        [1] Row
            [0] Spacer(width=450)
            [1] Toggle(name='SHOW S', width=200)
        [2] Row
            [0] Spacer(width=450)
            [1] IntSlider(end=30, name='y-max', start=6, step=3, value=6, width=200)
        [3] Row
            [0] Spacer(width=450)
            [1] IntSlider(end=60, name='t-max', start=15, step=5, value=25, width=200)

- The grey dotted lines are the x and y value computed from the intersection of the nullclines, respectively $n_0$ and $d_0$. If we zoom in to the crossing of the nullclines, we see that n is at ~3.98 which is the higher steady state that we fall out of. 

- Note that we do not see a second crossing for the later steady state in the nullcline plot bc we are plotting only the *homogeneous* steady state. 

- We can see that contrary to the adjacent cell 1 and cell 2 model, we can have unstable fixed points by adding in this repressive action of S on D. Is this actually the case? To answer this question, the **`SHOW S` toggle** was added to the trajectories above. We can see that at the moment the homogeneous steady state becomes unstable and bifurcates, $s1$ and $s2$ bifurcate as well (sorry $s2$ gets chopped off and looks like it grows unboundedly, but you can slide `y-max` to 30 to see that $s2$ stabilizes). 

- Bifurcations are allowed to appear when the first derivative vanishes. To be honest, I forgot a lot of the details of bifurcation analysis, but I remember that odd functions demonstrate pitchfork bifurcations, but the $\beta$ and inactivation terms here don't give odd functions. In a Hopf bifurcation, the real part of the eigenvalues are switching signs, require us to have complex eigenvalues to begin with. 
- Let's computationally solve for the eigenvalues, just to see what's happening:  
    Note that each row of our transpose Jacobian now has an extra $\kappa_s d_0$ or $\kappa_s n_0$ term. 

\begin{align}
\hspace{0.1em}\\[1em]
A = \begin{pmatrix}
-(1+d_0 (\kappa+1))  &  0 & -\kappa n_0  &  n_0  &  0  &  0 \\
0  &  -(1+d_0 (\kappa+1)) & -n_0  & -\kappa n_0  &  0  &  0 \\ 
- \kappa d_0 & - d_0 & -(1+n_0 (\kappa + 1)) & 0 & 2\beta_d s_0 / (1+s_0^2)^2 & 0 \\
- d_0 & - \kappa d_0 & 0 & -(1+n_0 (\kappa + 1))& 0 & 2\beta_d s_0 / (1+s_0^2)^2 \\
\kappa_s d_0 & 0 & 0 & \kappa_s n_0 & -\gamma & 0 \\
0 & \kappa_s d_0 & \kappa_s n_0 & 0 & 0 & -\gamma\\
\end{pmatrix}
\end{align}

In [48]:
def eigens(no, do, beta_d, gamma, kappa, kappa_s):
    bd = beta_d
    g = gamma
    k = kappa
    ks = kappa_s
    so = ks / g * no * do
    
    A = np.array([
        [-(1+do*(k+1)), 0, -k*no, -no, 0, 0],
        [0,-(1+do*(k+1)), -no, -k*no, 0, 0],
        [-k*do, -do, -(1+no*(k+1)), 0, 2*bd*so/(1+so**2)**2,0],
        [-do, -k*do, 0, -(1+no*(k+1)), 0, 2*bd*so/(1+so**2)**2],
        [ks*do, 0, 0, ks*no, -g, 0],
        [0, ks*do, ks*no, 0, 0, -g]
    ])
    return np.linalg.eigvals(A)

In [49]:
no_st = 3.9823664
do_st = 0.1054527

beta_d = 18
gamma = 5
kappa = 10
kappa_s = 20

eigvals = eigens(no_st, do_st, beta_d, gamma, kappa, kappa_s)
print("Eigenvalues: ")
__ = [print(np.round(_, 2)) for _ in eigvals]

Eigenvalues: 
-52.86
-35.24
-14.01
-2.71
-1.0
1.89


So one eigenvalue destabilizes the whole thing! This branching behavior makes sense since we then have a multi-dimensional **saddle** at the fixed point. This motivates me to label this as a **saddle-node bifurcation**.

# lattice model 🕸 🐝 🕸 
<hr>
I really wanted to implement the lattice grid model I saw in lecture. Hard coded everything! Kinda regret not making some more hexagons. 

\begin{align}
\cfrac{\mathrm{d}N_i}{\mathrm{d}t} &= \beta_N - \gamma N_i - \cfrac{N_i \langle D_j \rangle_i}{k_t} - \cfrac{N_i D_i}{k_c} \\[0.5em]
\cfrac{\mathrm{d}D_i}{\mathrm{d}t} &= \beta_D \cfrac{1}{1+S_i^m} - \gamma D_i - \cfrac{D_i \langle N_j \rangle_i}{k_t} - \cfrac{N_i D_i}{k_c} \\[0.5em]
\cfrac{\mathrm{d}S_i}{\mathrm{d}t} &= \beta_S \cfrac{(N_i \langle D_j \rangle _i)^n}{k_{RS}+(N_i \langle D_j\rangle_i)^n}- \gamma_S S_i
\end{align}

We start by constructing a grid of 60 cells, and making an adjacency dictionary so that we can easily retrieve the relevant neighbors when calculating the average concentrations.

In [50]:
_adjacencies = {
    1: [2, 11, 12],
    2: [1, 3, 12, 13],
    3: [2, 4, 13, 14],
    4: [3, 5, 14, 15],
    5: [4, 6, 15, 16],
    6: [5, 7, 16, 17],
    7: [6, 8, 17, 18],
    8: [7, 9, 18, 19],
    9: [8, 10, 19, 20],
    10: [9, 20],
    11: [1, 12, 21],
    12: [1, 2, 11, 13, 21, 22],
    13: [2, 3, 12, 14, 22, 23],
    14: [3, 4, 13, 15, 23, 24],
    15: [4, 5, 14, 16, 24, 25],
    16: [5, 6, 15, 17, 25, 26],
    17: [6, 7, 16, 18, 26, 27],
    18: [7, 8, 17, 19, 27, 28],
    19: [8, 9, 18, 20, 28, 29], 
    20: [9, 10, 19, 29, 30],
    21: [11, 12, 22, 31, 32],
    22: [12, 13, 21, 23, 32, 33],
    23: [13, 14, 22, 24, 33, 34],
    24: [14, 15, 23, 25, 34, 35],
    25: [15, 16, 24, 26, 35, 36],
    26: [16, 17, 25, 27, 36, 37],
    27: [17, 18, 26, 28, 37, 38],
    28: [18, 19, 27, 29, 38, 39],
    29: [19, 20, 28, 30, 39, 40],
    30: [20, 29, 40],
    31: [21, 32, 41],
    32: [21, 22, 31, 33, 41, 42],
    33: [22, 23, 32, 34, 42, 43],
    34: [23, 24, 33, 35, 43, 44],
    35: [24, 25, 34, 36, 44, 45],
    36: [25, 26, 35, 37, 45, 46],
    37: [26, 27, 36, 38, 46, 47],
    38: [27, 28, 37, 39, 47, 48],
    39: [28, 29, 38, 40, 48, 49],
    40: [29, 30, 39, 49, 50],
    41: [31, 32, 42, 51, 52],
    42: [32, 33, 41, 43, 52, 53],
    43: [33, 34, 42, 44, 53, 54],
    44: [34, 35, 43, 45, 54, 55],
    45: [35, 36, 44, 46, 55, 56],
    46: [36, 37, 45, 47, 56, 57],
    47: [37, 38, 46, 48, 57, 58],
    48: [38, 39, 47, 49, 58, 59],
    49: [39, 40, 48, 50, 59, 60],
    50: [40, 49, 60],
    51: [41, 52], 
    52: [41, 42, 51, 53],
    53: [42, 43, 52, 54],
    54: [43, 44, 53, 55],
    55: [44, 45, 54, 56],
    56: [45, 46, 55, 57],
    57: [46, 47, 56, 58],
    58: [47, 48, 57, 59],
    59: [48, 49, 58, 60],
    60: [49, 50, 59]
}

adjacencies = {}
for _k, _v in _adjacencies.items():
    k = _k -1
    v = [_-1 for _ in _v]
    adjacencies[k] = v

In [51]:
def derivs(X, t, beta_N, beta_D, beta_S, gamma, gamma_S, k_c, k_t, k_RS, m, n):
    A = np.empty(180)
    for _ in range(0, 60):      # dN_dt
        Ni = X[_]
        Di = X[_+60]
        Di_avg = np.mean([X[neighbor+60] for neighbor in adjacencies[_]])
        A[_] = beta_N - gamma*Ni - Ni*Di_avg/k_t - Ni*Di/k_c
    
    for _ in range(60, 120):    # dD_dt
        Ni = X[_-60]
        Ni_avg = np.mean([X[neighbor] for neighbor in adjacencies[_-60]])
        Di = X[_]
        Si = X[_+60]
        A[_] = beta_D/(1+Si**m) - gamma*Di - Di*Ni_avg/k_t - Ni*Di/k_c
        
    for _ in range(120, 180):   # dS_dt
        Ni = X[_-120]
        Di_avg = np.mean([X[neighbor+60] for neighbor in adjacencies[_-120]]) 
        Si = X[_]
        A[_] = beta_S*(Ni*Di_avg)**n / (k_RS + (Ni*Di_avg)**n) - gamma_S*Si
        
    return A

In [52]:
beta_N = 50
beta_D = 100
n, m = 3, 3

beta_S = 5
gamma_S = 0.60
gamma = 10

k_c = 1
k_t = 1
k_RS = 1

t = np.linspace(0, 10, 500)
Xo = np.random.random(180) + 2

args = (beta_N, beta_D, beta_S, gamma, gamma_S, k_c, k_t, k_RS, m, n)
trajs = scipy.integrate.odeint(derivs, Xo, t, args).T

In [53]:
color_N, color_D = "#0A7F8C","#B6423D"
p = utilities.plotter_lattice_trajectories(t, trajs, color_N, color_D)
bokeh.io.show(p)

This notebook is getting a bit slow, so I made a movie of the grid of $[D]$ over time. We can see the spatial pattern emerging, where D is either high or really low, which is what we see above. I couldn't decide which color scheme to use so both are included. 

In [54]:
from IPython.display import Video
Video("all_out.mp4", width=500)

Here is the slider:

In [55]:
utilities.plotter_lattice_hexagons(t, trajs, color_D)

Column
    [0] IntSlider(end=499, name='time')
    [1] ParamFunction(function)

- So we see this pattern where the filled hexagons are surrounded by empty ones, which means that two neighboring cells can not both be senders, but receievers can be adjacent. This makes sense intuitively, since it would be confusing or counterproductive / a waste of time for adjacent signals to both influence a common neighbor. The system greatly dislikes the initialized random condition, and moves away from it very quickly. After running this simulation many times, the patterning does indeed shift with different seeds/i.c.'s, but the pattern itself nonetheless persists.   
- Because a lot of this was hard-coded and I am under some serious time constraints, I had this idea that we can use some geometric similarities and shifting boundary conditions to generalize this pattern across a larger grid (i.e. expanding the repeating patterns with different random initial conditions repeatedly solving for the same grid, then filling out the interior again and again). I realized the problem in this was that the boundary conditions would be ommitted, and that if I am truly interested in the dynamics, there is no negotiation with the symmetry other than more hard-coding of a larger grid.   
- This was very satisfying to see for myself, I was amazed by the results shown in class, and I am happy it is a reproducible result. Thank you so so much for reading, and to whoever long ago first dreamed of this late day business—they're a godsend!

# Appendix A: 
### Cis and Trans Complex Code

In [27]:
# def plotter_complexes():
#     D_plate_slider = pn.widgets.FloatSlider(name="D_plate", start=0.05, end=1.4, value=0.05, step=0.05, width=300)

#     @pn.depends(D_plate_slider.param.value)
#     def _plotter_complexes(D_plate):
#         D_trans = D_plate
#         No = beta_N / (gamma_N + k_c * Do + k_t * D_plate) 
#         Co = k_c * No * Do
#         To = k_t*(N_trans*Do + No*D_trans)
#         args = (beta_N, beta_D, beta_R, 
#             gamma_N, gamma_D, gamma_R, gamma_S, 
#             k_c, k_t, k_RS, p, 
#             N_trans, D_trans)
#         NDSRCTo = np.array([No, Do, So, Ro, Co, To])
#         t = np.linspace(0, 100, 1000)    
        
#         _NDSRCT = scipy.integrate.odeint(derivs_A_supp, NDSRCTo, t, args)
#         N_traj, D_traj, S_traj, R_traj, C_traj, T_traj = _NDSRCT.T    
        
#         q = bokeh.plotting.figure(
#             height=400, width=400, 
#             title="(In)Activation Complexes", 
#             x_axis_label="time (hrs)", y_axis_label="[complex]",
#             y_range=(-2, 55)
#         )
#         q.line(t, C_traj, line_color="#bbbbbb", line_width=3, legend_label="cis inactivated complexes") # inactivation
#         q.line(t, T_traj, line_color="#e3a201", line_width=3, legend_label="trans activated complexes")
#         q.legend.location = 'top_left'
#         q = style(q)
        
#         return q
#     lay_slider = pn.Row(D_plate_slider, align="center")
    
#     return pn.Column(lay_slider, _plotter_complexes)
    
    
# def plotter_shift():
#     beta_D = 0
#     D_plate = D_trans = 0.5

#     beta_N_slider = pn.widgets.FloatSlider(name="βN", start=0.5, end=1.8, value=1, step=0.1, width=150)
#     gamma_slider = pn.widgets.FloatSlider(name="γ", start=0.03, end=0.30, value=0.08, step=0.01, width=150)

#     @pn.depends(beta_N_slider.param.value, gamma_slider.param.value)
#     def plotter_shift(beta_N, gamma):
#         gamma_N = gamma_D = gamma
        
#         No = beta_N / (gamma_N + k_c * Do + k_t * D_plate) 
#         Co = k_c * No * Do
#         To = k_t*(N_trans*Do + No*D_trans)
#         args = (beta_N, beta_D, beta_R, 
#             gamma_N, gamma_D, gamma_R, gamma_S, 
#             k_c, k_t, k_RS, p, 
#             N_trans, D_trans)
#         NDSRCTo = np.array([No, Do, So, Ro, Co, To])
#         t = np.linspace(0, 100, 1000)    

#         _NDSRCT = scipy.integrate.odeint(derivs_A_supp, NDSRCTo, t, args)
#         N_traj, D_traj, S_traj, R_traj, C_traj, T_traj = _NDSRCT.T    

#         q = bokeh.plotting.figure(
#             height=450, width=400, 
#             title="(In)Activation Complexes", 
#             x_axis_label="time (hrs)", y_axis_label="[complex]",
#             y_range=(-2, 55)
#         )
#         q.line(t, C_traj, line_color="#bbbbbb", line_width=3) # inactivation
#         q.line(t, T_traj, line_color="#e3a201", line_width=3)

#         legend = bokeh.models.Legend(items=[("cis inactivated complexes", [q.line(line_width=3, line_color="#bbbbbb")]),
#                                             ("trans activated complexes", [q.line(line_width=3, line_color="#e3a201")])],
#                                      location='center'
#                                     )
#         q.add_layout(legend, 'below')
        
#         return style(q)

#     lay_params = pn.Row(pn.Spacer(width=50), beta_N_slider, gamma_slider)
#     return pn.Column(lay_params, plotter_shift, align="center")
    
    

# Appendix B: 
### Homogeneous Code

In [28]:
# # intersection code lifted from pip package `intersection`
# def _rect_inter_inner(x1, x2):
#     n1 = x1.shape[0]-1
#     n2 = x2.shape[0]-1
#     X1 = np.c_[x1[:-1], x1[1:]]
#     X2 = np.c_[x2[:-1], x2[1:]]
#     S1 = np.tile(X1.min(axis=1), (n2, 1)).T
#     S2 = np.tile(X2.max(axis=1), (n1, 1))
#     S3 = np.tile(X1.max(axis=1), (n2, 1)).T
#     S4 = np.tile(X2.min(axis=1), (n1, 1))
#     return S1, S2, S3, S4

# def _rectangle_intersection_(x1, y1, x2, y2):
#     S1, S2, S3, S4 = _rect_inter_inner(x1, x2)
#     S5, S6, S7, S8 = _rect_inter_inner(y1, y2)

#     C1 = np.less_equal(S1, S2)
#     C2 = np.greater_equal(S3, S4)
#     C3 = np.less_equal(S5, S6)
#     C4 = np.greater_equal(S7, S8)

#     ii, jj = np.nonzero(C1 & C2 & C3 & C4)
#     return ii, jj

# def intersection(x1, y1, x2, y2):
#     """
# INTERSECTIONS Intersections of curves.
#    Computes the (x,y) locations where two curves intersect.  The curves
#    can be broken with NaNs or have vertical segments.
# usage:
# x,y=intersection(x1,y1,x2,y2)
#     Example:
#     a, b = 1, 2
#     phi = np.linspace(3, 10, 100)
#     x1 = a*phi - b*np.sin(phi)
#     y1 = a - b*np.cos(phi)
#     x2=phi
#     y2=np.sin(phi)+2
#     x,y=intersection(x1,y1,x2,y2)
#     plt.plot(x1,y1,c='r')
#     plt.plot(x2,y2,c='g')
#     plt.plot(x,y,'*k')
#     plt.show()
#     """
#     x1 = np.asarray(x1)
#     x2 = np.asarray(x2)
#     y1 = np.asarray(y1)
#     y2 = np.asarray(y2)

#     ii, jj = _rectangle_intersection_(x1, y1, x2, y2)
#     n = len(ii)

#     dxy1 = np.diff(np.c_[x1, y1], axis=0)
#     dxy2 = np.diff(np.c_[x2, y2], axis=0)

#     T = np.zeros((4, n))
#     AA = np.zeros((4, 4, n))
#     AA[0:2, 2, :] = -1
#     AA[2:4, 3, :] = -1
#     AA[0::2, 0, :] = dxy1[ii, :].T
#     AA[1::2, 1, :] = dxy2[jj, :].T

#     BB = np.zeros((4, n))
#     BB[0, :] = -x1[ii].ravel()
#     BB[1, :] = -x2[jj].ravel()
#     BB[2, :] = -y1[ii].ravel()
#     BB[3, :] = -y2[jj].ravel()

#     for i in range(n):
#         try:
#             T[:, i] = np.linalg.solve(AA[:, :, i], BB[:, i])
#         except:
#             T[:, i] = np.Inf

#     in_range = (T[0, :] >= 0) & (T[1, :] >= 0) & (
#         T[0, :] <= 1) & (T[1, :] <= 1)

#     xy0 = T[2:, in_range]
#     xy0 = xy0.T
#     return xy0[:, 0], xy0[:, 1]


# def plotter_homogeneous(color_n1='#5779a3', color_n2='#84b5b2', color_d1='#d1605e', color_d2='#e59244', color_s1='#808bbe', color_s2='#b09dc9'):
#     alpha = 2
#     no, do = 5, 100
#     n1o, n2o = no, no
#     d1o, d2o = do, do
#     s1o, s2o = 0, 0
#     n1n2d1d2s1s2_o = np.array([n1o, n2o, d1o, d2o, s1o, s2o])
    
#     beta_n_slider = pn.widgets.FloatSlider(name='βn', start=0.1, end=10, step=0.1, value=8.60, width=95)
#     beta_d_slider = pn.widgets.IntSlider(name='βd', start=1, end=30, step=1, value=18, width=95)
#     gamma_slider = pn.widgets.FloatSlider(name='γ', start=1, end=20, step=1, value=5, width=95)
#     kappa_slider = pn.widgets.FloatSlider(name='κ', start=1, end=10, step=0.5, value=10, width=95)
#     kappa_s_slider = pn.widgets.FloatSlider(name='κS', start=1, end=100, step=2, value=20, width=95)

#     show_S_toggle = pn.widgets.Toggle(name="SHOW S", value=False, width=200)
#     t_max_slider = pn.widgets.IntSlider(name="t-max", start=15, end=60, step=5, value=25, width=200)
#     y_max_slider = pn.widgets.IntSlider(name="y-max", start=6, end=30, step=3, value=6, width=200)

#     @pn.depends(beta_n_slider.param.value, beta_d_slider.param.value, 
#                 gamma_slider.param.value, kappa_slider.param.value, kappa_s_slider.param.value,
#                 y_max_slider.param.value, t_max_slider.param.value, show_S_toggle.param.value,
#                )
#     def _plotter_find_params(beta_n, beta_d, gamma, kappa, kappa_s, y_max, t_max, show_S):
#         do_space = np.logspace(-5, 5, 200)
#         no_space = np.logspace(-5, 5, 200)

#         # ..... NULLCLINES .... 
#         no = beta_n / (1+do_space*(1+kappa))
#         do = beta_d / (1+(kappa_s/(gamma*(1+kappa))*(beta_n - no_space))**alpha) / (1+no_space*(1+kappa))
        
#         # finding fixed point intersection
#         x, y = intersection(no_space, do, no, do_space)
        
        
#         p = bokeh.plotting.figure(
#                 height=325, width=375, 
#                 x_axis_type='log', y_axis_type='log',
#                 x_range=(1e-3, 1e3), y_range=(1e-3, 1e3),
#                 x_axis_label="n", y_axis_label="d",
#                 title='Nullclines',
#             )
#         t = np.linspace(0, t_max, 300)
#         p.line(no_space, do, line_width=3, line_color='#3d2314')
#         p.line(no, do_space, line_width=3, line_color='#3d2314')
#         p.circle(x, y, size=13, line_color='#3d2314', fill_color='white', line_width=4)

#         # .... TRAJECTORIES .... 
#         args = (alpha, beta_n, beta_d, gamma, kappa, kappa_s)
#         _n1n2d1d2s1s2 = scipy.integrate.odeint(derivs_C, n1n2d1d2s1s2_o, t, args)
#         n1_traj, n2_traj, d1_traj, d2_traj, s1_traj, s2_traj = _n1n2d1d2s1s2.T
#         q = bokeh.plotting.figure(
#             height=325, width=475,
#             x_axis_label='time', y_axis_label='[ ]',
#             title='Trajectories',
#             y_range=(-0.2, y_max+0.2)
#         )
#         grey = "#bbbbbb"

#         if show_S:
#             q.line((t[0], t[-1]), (x, x), line_width=2, line_color=grey, line_dash='dotdash')
#             q.line((t[0], t[-1]), (y, y), line_width=2, line_color=grey, line_dash='dotdash')
#             q.line(t, n1_traj, line_width=2, line_color=grey)
#             q.line(t, n2_traj, line_width=2, line_color=grey)
#             q.line(t, d1_traj, line_width=2, line_color=grey)
#             q.line(t, d2_traj, line_width=2, line_color=grey)
#             q.line(t, s2_traj, line_width=4, line_color=color_s2)
#             q.line(t, s1_traj, line_width=4, line_color=color_s1)
#             legend = bokeh.models.Legend(items=[
#                 ("s1", [q.line(line_width=3, line_color=color_s1)]),
#                 ("s2", [q.line(line_width=3, line_color=color_s2)]),
#             ], location='center')
#         else: 
#             q.line((t[0], t[-1]), (x, x), line_width=4, line_color=grey, line_dash='dotdash')
#             q.line((t[0], t[-1]), (y, y), line_width=4, line_color=grey, line_dash='dotdash')

#             q.line(t, n1_traj, line_width=3, line_color=color_n1)
#             q.line(t, n2_traj, line_width=3, line_color=color_n2)
#             q.line(t, d1_traj, line_width=3, line_color=color_d1)
#             q.line(t, d2_traj, line_width=3, line_color=color_d2)
#             legend = bokeh.models.Legend(items=[
#                 ("n1", [q.line(line_width=3, line_color=color_n1)]),
#                 ("n2", [q.line(line_width=3, line_color=color_n2)]),
#                 ("d1", [q.line(line_width=3, line_color=color_d1)]),
#                 ("d2", [q.line(line_width=3, line_color=color_d2)]),
#                 ("fp", [q.line(line_width=3, line_color=grey, line_dash="dotdash")])
#             ], location='center')
            
#         q.add_layout(legend, 'right')
        
#         return pn.Row(style(p), style(q))

#     lay_params = pn.Column(
#         pn.Spacer(height=10),
#         beta_n_slider, beta_d_slider, gamma_slider, 
#         kappa_slider, kappa_s_slider
#     )
#     lay_traj = pn.Column(
#         _plotter_find_params, 
#         pn.Row(pn.Spacer(width=450), show_S_toggle),
#         pn.Row(pn.Spacer(width=450), y_max_slider),
#         pn.Row(pn.Spacer(width=450), t_max_slider),
#     )
#     return pn.Row(lay_params, lay_traj)


# Appendix C: 
### Lattice Code

In [29]:
# def plotter_lattice_trajectories(t, trajs, color_N, color_D):
#     p = bokeh.plotting.figure(
#         height=400, width=575,
#         title="Lattice Trajectories",
#         x_axis_label="time",
#         y_axis_label="[ ]"
#     )
#     lw = 2

#     for traj in trajs[:60]: 
#         p.line(t, traj, line_width=lw, line_color=color_N, line_alpha=0.5)
#     for traj in trajs[60:120]: 
#         p.line(t, traj, line_width=lw, line_color=color_D, line_alpha=0.5)
#     legend = bokeh.models.Legend(items=[
#         ("N", [p.line(line_color=color_N, line_width=lw)]),
#         ("D", [p.line(line_color=color_D, line_width=lw)]), 
#     ], location='center')
#     p.add_layout(legend, 'right')
#     return style(p)
    
    
# def hex_to_rgb(palette):
#     if type(palette)==str: return tuple([int(palette.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)])
#     return [tuple(int(h.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) for h in palette]

# def rgb_to_hex(palette):
#     if type(palette[0])==int: return "#%02x%02x%02x" % palette
#     return ["#%02x%02x%02x" % (r,g,b) for (r,g,b) in palette]

# def get_D_colors(trajs, color_D):
#     color_D_rgb = hex_to_rgb(color_D)
#     N_trajs, D_trajs, S_trajs = trajs[:60], trajs[60:120], trajs[120:]
#     D_max = max(D_trajs.flatten())

#     color_D_rgb = hex_to_rgb(color_D)

#     D_colors, D_alphas = [], []
#     for D_traj in D_trajs:
#         _D_colors, _D_alphas = [], []
#         for D in D_traj:
#             f = D / D_max
#             _color = f
#             _color_rgb = [int(_) for _ in f * np.array(color_D_rgb)]
#             _color = rgb_to_hex(tuple(_color_rgb))
#             _D_colors.append(_color)
#             _D_alphas.append(f)
#         D_colors.append(_D_colors)
#         D_alphas.append(_D_alphas)
        
#     return D_colors, D_alphas

# def rearrange(D_time):
#     rearranged = []
#     for i in range(10):
#         for j in range(i, i+60, 10):
#             rearranged.append(D_time[j])
#     return rearranged
    


# def plotter_lattice_hexagons(t, trajs, color_D):
#     D_colors, D_alphas = get_D_colors(trajs, color_D)
#     time_slider = pn.widgets.IntSlider(name="time", start=0, end=len(t)-1, value=0)
    
#     r = [_ for _ in range(6)]*10
#     _q = [0, -1, -1, -2, -2, -3]
#     __q = [[_ + __ for _ in _q] for __ in range(10)]
#     q = [_ for __ in __q for _ in __]
    
#     @pn.depends(time_slider.param.value)
#     def _plotter_lattice_hexagons(time):
#         D_time_colors = [D_traj[time] for D_traj in D_colors]
#         D_time_alphas = [D_traj[time] for D_traj in D_alphas]
        
#         colors = rearrange(D_time_colors)
#         alphas = rearrange(D_time_alphas)

#         source = bokeh.models.ColumnDataSource(dict(r=r, q=q, color=colors, alpha=alphas))
        
#         viz1 = bokeh.models.Plot(plot_width=600, plot_height=300, toolbar_location=None)
#         viz2 = bokeh.models.Plot(plot_width=600, plot_height=300, toolbar_location=None)
#         glyph1 = bokeh.models.HexTile(
#             q='q',r='r', 
#             line_color="white", 
#             fill_color="color", 
#         )
#         glyph2 = bokeh.models.HexTile(
#             q='q',r='r', 
#             fill_color=color_D,
#             fill_alpha="alpha",
#             line_color="black", 
#         )
#         viz1.add_glyph(source, glyph1)
#         viz2.add_glyph(source, glyph2)
#         return pn.Column(viz2, viz1)

#     return pn.Column(time_slider, _plotter_lattice_hexagons)